In [34]:
spacex_launch_df = pd.read_csv('~\spacex_launch_dash.csv')
app = Dash(__name__)

In [35]:
spacex_launch_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [36]:
spacex_launch_df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [70]:
import dash
from dash import dcc, html
import plotly.graph_objects as go
import pandas as pd

spacex_launch_df = pd.read_csv('~\spacex_launch_dash.csv')

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Total Success Launches by Site"),
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'All'}
        ] + [{'label': site, 'value': site} for site in spacex_launch_df['Launch Site'].unique()],
        value='All'
    ),
    dcc.Graph(id='success-pie-chart')
])

@app.callback(
    dash.Output('success-pie-chart', 'figure'),
    dash.Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'All':
        red_df = spacex_launch_df
    else:
        red_df = spacex_launch_df[spacex_launch_df['Launch Site'] == selected_site]

    success_counts = red_df['class'].value_counts()

    labels = ['Success', 'Failure']
    values = [success_counts.get(1, 0), success_counts.get(0, 0)]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
    fig.update_layout(title_text=f"Success for {selected_site}")
    return fig

if __name__ == '__main__':
    app.run(mode='inline')

In [82]:
import plotly.express as px

min_val = spacex_launch_df['Payload Mass (kg)'].min()
max_val = spacex_launch_df['Payload Mass (kg)'].max()

app = dash.Dash(__name__)

fig = px.scatter(spacex_launch_df, x="Payload Mass (kg)", y="class", color="Booster Version Category", 
                 title="Correlation between Success and Payload for All Sites")

app.layout = html.Div([
    html.H1("Correlation between Success and Payload for All Sites"),
    dcc.RangeSlider(
        id='id',
        min=0, max=10000, step=1000,
        marks={0: '0', 100: '100'},
        value=[min_val, max_val]),
    dcc.Graph(figure=fig)
])

if __name__ == '__main__':
    app.run(mode='inline')